## Import Statements

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import OneHotEncoder
from scipy.sparse import hstack

from xgboost import XGBRegressor

In [2]:
df = pd.read_csv("train.csv")
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [3]:
min_date=df["date_id"].min()
max_date=df["date_id"].max()
min_date, max_date

(0, 480)

In [4]:
min_time=df["time_id"].min()
max_time=df["time_id"].max()
min_time, max_time
# What is the significance of time_id?

(0, 26454)

In [5]:
features = [col for col in df.columns]
features

['stock_id',
 'date_id',
 'seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'target',
 'time_id',
 'row_id']

## Data Cleaning Approach:
1. Detection and Handling of Missing values
2. Detection and Handling of Outliers
3. Feature Scaling and Mean Normalization
4. Feature Engineering

### 1. Detection and Handling of Missing values

In [6]:
df["target"].isnull().sum()

88

In [7]:
for col in features:
    num = df[col].isnull().sum()
    print(f'{col} has {num} nan values')

# There are 88 missing targets: Remove these data points
df_nan_target_dropped=df[df["target"].notna()].copy()
print("After removing nan targets")
df_nan_target_dropped["target"].isnull().sum()
for col in features:
    num = df_nan_target_dropped[col].isnull().sum()
    print(f'{col} has {num} nan values')

stock_id has 0 nan values
date_id has 0 nan values
seconds_in_bucket has 0 nan values
imbalance_size has 220 nan values
imbalance_buy_sell_flag has 0 nan values
reference_price has 220 nan values
matched_size has 220 nan values
far_price has 2894342 nan values
near_price has 2857180 nan values
bid_price has 220 nan values
bid_size has 0 nan values
ask_price has 220 nan values
ask_size has 0 nan values
wap has 220 nan values
target has 88 nan values
time_id has 0 nan values
row_id has 0 nan values
After removing nan targets
stock_id has 0 nan values
date_id has 0 nan values
seconds_in_bucket has 0 nan values
imbalance_size has 132 nan values
imbalance_buy_sell_flag has 0 nan values
reference_price has 132 nan values
matched_size has 132 nan values
far_price has 2894254 nan values
near_price has 2857092 nan values
bid_price has 132 nan values
bid_size has 0 nan values
ask_price has 132 nan values
ask_size has 0 nan values
wap has 132 nan values
target has 0 nan values
time_id has 0 nan v

2nd type of nan values: imbalance_size, reference price, matched_size, bid_price, ask_price, wap

In [8]:
df_subset = df[df["imbalance_size"].isnull()]
df_subset.head()
print(df_subset["stock_id"].value_counts())

131    55
101    55
158    55
19     55
Name: stock_id, dtype: int64


For these 4 stocks, there are some missing data under these features. Let us keep track of these missing values using an indicator variable.

In [9]:
col_subset = ["imbalance_size", "reference_price", "matched_size", "bid_price", "ask_price", "wap"]

for col in col_subset:
    df_nan_target_dropped[col + '_missing'] = df_nan_target_dropped[col].isnull().astype(int)
df_nan_target_dropped[(df_nan_target_dropped["stock_id"]==131) & (df_nan_target_dropped["imbalance_size"].isnull())]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,wap,target,time_id,row_id,imbalance_size_missing,reference_price_missing,matched_size_missing,bid_price_missing,ask_price_missing,wap_missing
375268,131,35,300,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,90.130570,1955,35_300_131,1,1,1,1,1,1
375460,131,35,310,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,3.499985,1956,35_310_131,1,1,1,1,1,1
375652,131,35,320,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.239777,1957,35_320_131,1,1,1,1,1,1
375844,131,35,330,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.979900,1958,35_330_131,1,1,1,1,1,1
376036,131,35,340,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.050068,1959,35_340_131,1,1,1,1,1,1
376228,131,35,350,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,2.340078,1960,35_350_131,1,1,1,1,1,1
376420,131,35,360,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.389452,1961,35_360_131,1,1,1,1,1,1
376612,131,35,370,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.330210,1962,35_370_131,1,1,1,1,1,1
376804,131,35,380,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,-4.479885,1963,35_380_131,1,1,1,1,1,1
376996,131,35,390,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,-3.659725,1964,35_390_131,1,1,1,1,1,1


In [10]:
df_nan_target_dropped.fillna(method='ffill', inplace=True)
df_nan_target_dropped.fillna(method='bfill', inplace=True)

In [11]:
cat_variable = 'imbalance_buy_sell_flag'
features.remove(cat_variable)
num_variables = features
cat_variable, num_variables


('imbalance_buy_sell_flag',
 ['stock_id',
  'date_id',
  'seconds_in_bucket',
  'imbalance_size',
  'reference_price',
  'matched_size',
  'far_price',
  'near_price',
  'bid_price',
  'bid_size',
  'ask_price',
  'ask_size',
  'wap',
  'target',
  'time_id',
  'row_id'])

In [12]:
cat_df =df_nan_target_dropped[cat_variable].copy().values.reshape(-1, 1)
num_df = df_nan_target_dropped[num_variables].copy()

encoder = OneHotEncoder(drop='first')
cat_encoded = encoder.fit_transform(cat_df).toarray()
cat_encoded.shape

(5237892, 2)

In [13]:
cat_encoded = pd.DataFrame(cat_encoded, columns=["no_imbalance", "buy_side_imbalance"])
X = pd.concat([num_df, cat_encoded], axis=1)
X.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,no_imbalance,buy_side_imbalance
0,0.0,0.0,0.0,3180602.69,0.999812,13380276.64,1.000241,1.000241,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0.0,0_0_0,0.0,1.0
1,1.0,0.0,0.0,166603.91,0.999896,1642214.25,1.000241,1.000241,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0.0,0_0_1,0.0,0.0
2,2.0,0.0,0.0,302879.87,0.999561,1819368.03,1.000241,1.000241,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0.0,0_0_2,0.0,0.0
3,3.0,0.0,0.0,11917682.27,1.000171,18389745.62,1.000241,1.000241,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0.0,0_0_3,0.0,0.0
4,4.0,0.0,0.0,447549.96,0.999532,17860614.95,1.000241,1.000241,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0.0,0_0_4,0.0,0.0


### Perform some trial testing

In [ ]:
model = XGBRegressor()

In [91]:
is_mock = True

In [93]:
from public_timeseries_testing_util import make_env
env = make_env()
iter_test = env.iter_test()
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    sample_prediction['target'] = model.predict(test.drop(columns=["row_id", "date_id", "far_price", "near_price"]))
    env.predict(sample_prediction)
    counter += 1